In [1]:
!pip install requests
!pip install openai
!pip install tiktoken
!pip install pandas
!pip install matplotlib
!pip install plotly
!pip install scipy
!pip install scikit-learn
!pip install SQLAlchemy

#### Drafting Data Models

In [25]:
from sqlalchemy import create_engine, String, Text, Column, Enum, Integer, Float, Boolean, DateTime, ARRAY, JSON, ForeignKey, LargeBinary
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

# Models

Base = declarative_base()

class ProtocolSpace(Base):
    __tablename__ = 'protocol_space'
    id = Column(String, primary_key=True)
    name = Column(String)
    member = Column(ARRAY(Integer), nullable=False)
    avatar = Column(String)
    symbol = Column(String)
    verified = Column(Boolean)
    plugin = Column(JSON, nullable=True)
    space_id = Column(Integer)
    protocol_proposals = relationship("ProtocolProposals", back_populates="protocol_space")

class ProtocolProposals(Base):
    __tablename__ = 'protocol_proposals'
    id = Column(String, primary_key=True)
    ipfs = Column(String)
    title = Column(String)
    body = Column(Text)
    start = Column(DateTime)
    end = Column(DateTime)
    state = Column(Enum("closed", "active"))
    author = Column(String)
    created = Column(DateTime)
    choices = Column(ARRAY(String))
    scores_state = Column(Enum("final", "pending"))
    scores_total = Column(Float, nullable=True)
    scores = Column(ARRAY(Float))
    votes = Column(Integer)
    quorum = Column(Integer)
    symbol = Column(String)
    flagged = Column(Boolean)
    space_id = Column(Integer, ForeignKey('protocol_space.space_id'), primary_key=False)
    embedding_id = Column(Integer, ForeignKey('proposal_embeddings.embedding_id'), primary_key=False)
    protocol_space = relationship("ProtocolSpace", back_populates="protocol_proposals")
    protocol_votes = relationship("ProposalVotes", back_populates="protocol_proposals")
    proposal_embeddings = relationship("ProposalEmbeddings", back_populates="protocol_proposals", foreign_keys="ProposalEmbeddings.proposal_id")

class ProposalVotes(Base):
    __tablename__ = 'proposal_votes'
    id = Column(String, primary_key=True)
    voter = Column(String)
    created = Column(DateTime)
    choice = Column(Integer)
    space = Column(JSON)
    space_id = Column(Integer, ForeignKey('protocol_space.space_id'), primary_key=False)
    proposal_id = Column(String, ForeignKey('protocol_proposals.id'), primary_key=False)
    protocol_proposals = relationship("ProtocolProposals", back_populates="protocol_votes")


class ProposalEmbeddings(Base):
    __tablename__ = 'proposal_embeddings'
    embedding_id = Column(Integer, primary_key=True)
    proposal_id = Column(String, ForeignKey('protocol_proposals.id'), primary_key=False)
    embedding = Column(LargeBinary)
    protocol_proposals = relationship("ProtocolProposals", back_populates="proposal_embeddings", foreign_keys="ProposalEmbeddings.proposal_id", remote_side="ProtocolProposals.id")


protocol_space_instance = ProtocolSpace()
protocol_proposals_instance = ProtocolProposals()
proposal_votes_instance = ProposalVotes()
proposal_embeddings_instance = ProposalEmbeddings()

print(protocol_space_instance)
print(protocol_proposals_instance)
print(proposal_votes_instance)
print(proposal_embeddings_instance)

/var/folders/v0/71tqdwzx1_11vzpcg1y_4mj80000gn/T/ipykernel_7085/1820121725.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


#### proposal_df and space_df

In [26]:
import requests
import pandas as pd
from sqlalchemy import create_engine, String, Text, Column, Enum, Integer, Float, Boolean, DateTime, ARRAY, JSON, ForeignKey, LargeBinary
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

# Define your SQLAlchemy models (ProtocolSpace and ProtocolProposals) here.

# Initialize variables
url = "https://hub.snapshot.org/graphql"
skip = 0
first = 6
space_in = ["lido-snapshot.eth"]
state = "all"

# Initialize lists to store data
proposal_data = []
space_data = []

# Infinite loop for pagination
while True:
    # GraphQL query parameters
    payload = {
        "operationName": "Proposals",
        "variables": {
            "first": first,
            "skip": skip,
            "space_in": space_in,
            "state": state,
            "author_in": [],
            "title_contains": "",
            "flagged": False,
        },
        "query": """query Proposals($first: Int!, $skip: Int!, $state: String!, $space: String, $space_in: [String], $author_in: [String], $title_contains: String, $space_verified: Boolean, $flagged: Boolean) {
                      proposals(
                        first: $first,
                        skip: $skip,
                        where: {space: $space, state: $state, space_in: $space_in, author_in: $author_in, title_contains: $title_contains, space_verified: $space_verified, flagged: $flagged}
                      ) {
                        id
                        ipfs
                        title
                        body
                        start
                        end
                        state
                        author
                        created
                        choices
                        space {
                          id
                          name
                          members
                          avatar
                          symbol
                          verified
                          plugins
                        }
                        scores_state
                        scores_total
                        scores
                        votes
                        quorum
                        symbol
                        flagged
                      }
                    }""",
    }

    # Make the request
    response = requests.post(url, json=payload)
    data = response.json()

    # Extract proposals
    proposals = data.get("data", {}).get("proposals", [])

    # Break loop if no more proposals
    if not proposals:
        break

    for proposal in proposals:
        # Create instances of ProtocolSpace
        space_instance = ProtocolSpace(
            id=proposal["space"]["id"],
            name=proposal["space"]["name"],
            member=proposal["space"]["members"],
            avatar=proposal["space"]["avatar"],
            symbol=proposal["space"]["symbol"],
            verified=proposal["space"]["verified"],
            plugin=proposal["space"]["plugins"],
            space_id=None,  # You can set this to a specific value if needed
            protocol_proposals=[],
        )

        # Create instances of ProtocolProposals
        proposal_instance = ProtocolProposals(
            id=proposal["id"],
            ipfs=proposal["ipfs"],
            title=proposal["title"],
            body=proposal["body"],
            start=proposal["start"],
            end=proposal["end"],
            state=proposal["state"],
            author=proposal["author"],
            created=proposal["created"],
            choices=proposal["choices"],
            scores_state=proposal["scores_state"],
            scores_total=proposal["scores_total"],
            scores=proposal["scores"],
            votes=proposal["votes"],
            quorum=proposal["quorum"],
            symbol=proposal["symbol"],
            flagged=proposal["flagged"],
            space_id=space_instance.id,
            embedding_id=None,  # You can set this to a specific value if needed
            protocol_space=space_instance,
            protocol_votes=[],
            proposal_embeddings=[],
        )

        # Append instances to lists
        proposal_data.append(proposal_instance)
        space_data.append(space_instance)

    # Increment skip for pagination
    skip += first

# Create DataFrames from the collected data
proposal_df = pd.DataFrame([proposal.__dict__ for proposal in proposal_data])
space_df = pd.DataFrame([space.__dict__ for space in space_data])


print(proposal_df.columns)
print("\n")
print(space_df.columns)

Index(['_sa_instance_state', 'id', 'ipfs', 'title', 'body', 'start', 'end',
       'state', 'author', 'created', 'choices', 'scores_state', 'scores_total',
       'scores', 'votes', 'quorum', 'symbol', 'flagged', 'space_id',
       'embedding_id', 'protocol_space', 'protocol_votes',
       'proposal_embeddings'],
      dtype='object')


Index(['_sa_instance_state', 'id', 'name', 'member', 'avatar', 'symbol',
       'verified', 'plugin', 'space_id', 'protocol_proposals'],
      dtype='object')


#### push proposal_df & space_df into SQLite table


In [31]:
from sqlalchemy import create_engine, String, Text, Column, Enum, Integer, Float, Boolean, DateTime, JSON, ForeignKey, LargeBinary
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd
import json

# Define your SQLAlchemy models (ProtocolSpace, ProtocolProposals, ProposalEmbeddings, ProposalChoices, and ProposalScores) here.
Base = declarative_base()

# ProposalEmbeddings class
class ProposalEmbeddings(Base):
    __tablename__ = 'proposal_embeddings'
    embedding_id = Column(Integer, primary_key=True)
    proposal_id = Column(String, ForeignKey('protocol_proposals.id'), primary_key=False)
    embedding = Column(LargeBinary)
    protocol_proposals = relationship("ProtocolProposals", back_populates="proposal_embeddings", foreign_keys="ProposalEmbeddings.proposal_id", remote_side="ProtocolProposals.id")

# ProtocolSpace class with a separate table for member elements
class ProtocolSpace(Base):
    __tablename__ = 'protocol_space'
    id = Column(String, primary_key=True)
    name = Column(String)
    avatar = Column(String)
    symbol = Column(String)
    verified = Column(Boolean)
    plugin = Column(JSON, nullable=True)
    space_id = Column(Integer)
    protocol_proposals = relationship("ProtocolProposals", back_populates="protocol_space")
    members = relationship("SpaceMember", back_populates="protocol_space")

# SpaceMember class to represent array elements
class SpaceMember(Base):
    __tablename__ = 'space_member'
    id = Column(Integer, primary_key=True)
    member = Column(String)
    protocol_space_id = Column(String, ForeignKey('protocol_space.id'))
    protocol_space = relationship("ProtocolSpace", back_populates="members")

# ProtocolChoices class to represent array elements
class ProposalChoices(Base):
    __tablename__ = 'proposal_choices'
    id = Column(Integer, primary_key=True)
    choice = Column(String)
    protocol_proposal_id = Column(String, ForeignKey('protocol_proposals.id'))
    protocol_proposals = relationship("ProtocolProposals", back_populates="choices")

# ProposalScores class to represent array elements
class ProposalScores(Base):
    __tablename__ = 'proposal_scores'
    id = Column(Integer, primary_key=True)
    score = Column(Float)
    protocol_proposal_id = Column(String, ForeignKey('protocol_proposals.id'))
    protocol_proposals = relationship("ProtocolProposals", back_populates="scores")

# ProtocolProposals class
class ProtocolProposals(Base):
    __tablename__ = 'protocol_proposals'
    id = Column(String, primary_key=True)
    ipfs = Column(String)
    title = Column(String)
    body = Column(Text)
    start = Column(DateTime)
    end = Column(DateTime)
    state = Column(Enum("closed", "active"))
    author = Column(String)
    created = Column(DateTime)
    scores_state = Column(Enum("final", "pending"))
    scores_total = Column(Float, nullable=True)
    votes = Column(Integer)
    quorum = Column(Integer)
    symbol = Column(String)
    flagged = Column(Boolean)
    space_id = Column(String, ForeignKey('protocol_space.id'), primary_key=False)  # Reference the 'id' column of ProtocolSpace
    embedding_id = Column(Integer, ForeignKey('proposal_embeddings.embedding_id'), primary_key=False)  # Make the foreign key nullable without enforcing this constraint at table creation; nullable=True
    protocol_space = relationship("ProtocolSpace", back_populates="protocol_proposals")
    protocol_votes = relationship("ProposalVotes", back_populates="protocol_proposals")
    proposal_embeddings = relationship("ProposalEmbeddings", back_populates="protocol_proposals", foreign_keys="ProposalEmbeddings.proposal_id")  # Define the relationship without enforcing foreign key constraints for now; foreign_keys="ProposalEmbeddings.proposal_id"
    choices = relationship("ProposalChoices", back_populates="protocol_proposals")
    scores = relationship("ProposalScores", back_populates="protocol_proposals")

# Create an SQLite database and tables
engine = create_engine('sqlite:///lido_snapshot.db')
Base.metadata.create_all(engine)

# Remove _sa_instance_state column if present
if '_sa_instance_state' in proposal_df.columns:
    proposal_df = proposal_df.drop(columns=['_sa_instance_state'])


# Remove _sa_instance_state column if present
if '_sa_instance_state' in space_df.columns:
    space_df = space_df.drop(columns=['_sa_instance_state'])

# Function to convert complex objects in DataFrame columns
def convert_complex_objects(value):
    if isinstance(value, list):
        return ', '.join(map(str, value))
    elif hasattr(value, 'id'):  # Assuming the object has an 'id' attribute
        return str(value.id)
    elif isinstance(value, str) and value.startswith('<__main__.ProtocolProposals object at'):
        # Extract and return some identifier from the string representation
        # Assuming the identifier can be extracted, else return None or a default value
        return extract_identifier_from_string(value)
    return value

def extract_identifier_from_string(value):
    # This is a placeholder; you'll need to replace it with actual logic
    # For example, if the object's string representation is like "<__main__.ProtocolProposals object at 0x12345>",
    # and you want to extract "0x12345" as the identifier:
    return value.split()[-1].strip('>').strip('0x')


def prepare_df_for_sqlite(df):
    for column in df.columns:
        # Skip transformation for 'protocol_proposals' column
        if column == 'protocol_proposals':
            continue
        # Convert datetime columns to string format
        elif pd.api.types.is_datetime64_any_dtype(df[column]):
            df[column] = df[column].dt.strftime('%Y-%m-%d %H:%M:%S')
        # Serialize JSON/dict data
        elif isinstance(df[column].iloc[0], dict):
            df[column] = df[column].apply(json.dumps)
        # Convert other complex types to string
        else:
            df[column] = df[column].apply(str)
    return df


# Prepare DataFrames for insertion
proposal_df = prepare_df_for_sqlite(proposal_df)
space_df = prepare_df_for_sqlite(space_df)

# Insert DataFrames into SQLite tables
try:
    proposal_df.to_sql('lido_proposals', engine, if_exists='replace', index=False)
    space_df.to_sql('lido_space', engine, if_exists='replace', index=False)
except Exception as e:
    print(f"An error occurred: {e}")

# Optional: Commit the changes
session = sessionmaker(bind=engine)()
session.commit()
session.close()

print(space_df['protocol_proposals'].head())
print("successful insertion")


0    17741e6b
1    17741e5c
2    17741fc7
3    1773e98d
4    1773ea11
Name: protocol_proposals, dtype: object
successful insertion


/var/folders/v0/71tqdwzx1_11vzpcg1y_4mj80000gn/T/ipykernel_7085/2502339412.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


#### proposal_votes_df

In [32]:
import requests
import pandas as pd

# Initialize variables
url = "https://hub.snapshot.org/graphql"
first = 6
space_in = ["lido-snapshot.eth"]
state = "all"

# Function to fetch proposals
def fetch_proposals(skip):
    # Your existing code to fetch proposals
    payload = {
        "operationName": "Proposals",
        "variables": {
            "first": first,
            "skip": skip,
            "space_in": space_in,
            "state": state,
            "author_in": [],
            "title_contains": "",
            "flagged": False,
        },
        "query": """query Proposals($first: Int!, $skip: Int!, $state: String!, $space: String, $space_in: [String], $author_in: [String], $title_contains: String, $space_verified: Boolean, $flagged: Boolean) {
                      proposals(
                        first: $first,
                        skip: $skip,
                        where: {space: $space, state: $state, space_in: $space_in, author_in: $author_in, title_contains: $title_contains, space_verified: $space_verified, flagged: $flagged}
                      ) {
                        id
                        ipfs
                        title
                        body
                        start
                        end
                        state
                        author
                        created
                        choices
                        space {
                          id
                          name
                          members
                          avatar
                          symbol
                          verified
                          plugins
                        }
                        scores_state
                        scores_total
                        scores
                        votes
                        quorum
                        symbol
                        flagged
                      }
                    }""",
    }

    # Make the request
    response = requests.post(url, json=payload)
    data = response.json()

    # Extract proposals
    proposals = data.get("data", {}).get("proposals", [])

    return proposals

# Function to fetch and create a proposal_votes_df for each proposal
def fetch_and_create_votes_df(proposal_id, proposal_votes_df):
    payload = {
        "operationName": "Votes",
        "variables": {"proposal": proposal_id},
        "query": """query Votes($proposal: String!) {
                      votes(
                        first: 1000,
                        where: {proposal: $proposal}
                      ) {
                        id
                        voter
                        created
                        choice
                        space {
                          id
                        }
                      }
                    }""",
    }

    # Make the request
    response = requests.post(url, json=payload)
    data = response.json()

    # Extract and create a proposal_votes_df
    votes = data.get("data", {}).get("votes", [])
    votes_df = pd.DataFrame(votes)
    
    # Initialize ProposalVotes instances with the appropriate attributes
    proposal_votes = [
        ProposalVotes(
            id=row["id"],
            voter=row["voter"],
            created=row["created"],
            choice=row["choice"],
            space=row["space"],
            space_id=None,  # Set the space_id appropriately
            proposal_id=proposal_id,
        )
        for index, row in votes_df.iterrows()
    ]

    # Append instances to the proposal_votes_df
    if proposal_votes_df is None:
        proposal_votes_df = pd.DataFrame([proposal_vote.__dict__ for proposal_vote in proposal_votes])
    else:
        proposal_votes_df = pd.concat([proposal_votes_df, pd.DataFrame([proposal_vote.__dict__ for proposal_vote in proposal_votes])], ignore_index=True)

    return proposal_votes_df

# Initialize lists to store data
proposal_data = []
space_data = []
proposal_votes_df = None  # Initialize proposal_votes_df as None

# Infinite loop for pagination
skip = 0
while True:
    proposals = fetch_proposals(skip)
    if not proposals:
        break

    for proposal in proposals:
        # Your existing code to create instances of ProtocolSpace and ProtocolProposals

        # Fetch and create proposal_votes_df for the proposal
        proposal_votes_df = fetch_and_create_votes_df(proposal_instance.id, proposal_votes_df)

        # Append instances to lists
        proposal_data.append(proposal_instance)
        space_data.append(space_instance)

    # Increment skip for pagination
    skip += first


# Remove _sa_instance_state column if present
if '_sa_instance_state' in proposal_votes_df.columns:
    proposal_votes_df = proposal_votes_df.drop(columns=['_sa_instance_state'])



print("PROPOSAL_DF :", proposal_df.columns)
print("\n")
print("SPACE_DF :", space_df.columns)
print("\n")
print("PROPOSAL VOTES_DF :", proposal_votes_df.columns)

PROPOSAL_DF : Index(['id', 'ipfs', 'title', 'body', 'start', 'end', 'state', 'author',
       'created', 'choices', 'scores_state', 'scores_total', 'scores', 'votes',
       'quorum', 'symbol', 'flagged', 'space_id', 'embedding_id',
       'protocol_space', 'protocol_votes', 'proposal_embeddings'],
      dtype='object')


SPACE_DF : Index(['id', 'name', 'member', 'avatar', 'symbol', 'verified', 'plugin',
       'space_id', 'protocol_proposals'],
      dtype='object')


PROPOSAL VOTES_DF : Index(['id', 'voter', 'created', 'choice', 'space', 'space_id', 'proposal_id'], dtype='object')


#### Insert proposal_votes_df into SQLite table: lido_votes

In [37]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import json
# ... [Include your other necessary imports and SQLAlchemy class definitions]

# Define your SQLAlchemy models
Base = declarative_base()

class ProposalVotes(Base):
    __tablename__ = 'proposal_votes'
    id = Column(String, primary_key=True)
    voter = Column(String)
    created = Column(DateTime)
    choice = Column(Integer)
    space = Column(JSON)
    space_id = Column(Integer, ForeignKey('protocol_space.space_id'), primary_key=False)
    proposal_id = Column(String, ForeignKey('protocol_proposals.id'), primary_key=False)
    protocol_proposals = relationship("ProtocolProposals", back_populates="protocol_votes")


# ProtocolProposals class
class ProtocolProposals(Base):
    __tablename__ = 'protocol_proposals'
    id = Column(String, primary_key=True)
    ipfs = Column(String)
    title = Column(String)
    body = Column(Text)
    start = Column(DateTime)
    end = Column(DateTime)
    state = Column(Enum("closed", "active"))
    author = Column(String)
    created = Column(DateTime)
    scores_state = Column(Enum("final", "pending"))
    scores_total = Column(Float, nullable=True)
    votes = Column(Integer)
    quorum = Column(Integer)
    symbol = Column(String)
    flagged = Column(Boolean)
    space_id = Column(String, ForeignKey('protocol_space.id'), primary_key=False)  # Reference the 'id' column of ProtocolSpace
    embedding_id = Column(Integer, ForeignKey('proposal_embeddings.embedding_id'), primary_key=False)  # Make the foreign key nullable without enforcing this constraint at table creation; nullable=True
    protocol_space = relationship("ProtocolSpace", back_populates="protocol_proposals")
    protocol_votes = relationship("ProposalVotes", back_populates="protocol_proposals")
    proposal_embeddings = relationship("ProposalEmbeddings", back_populates="protocol_proposals", foreign_keys="ProposalEmbeddings.proposal_id")  # Define the relationship without enforcing foreign key constraints for now; foreign_keys="ProposalEmbeddings.proposal_id"
    choices = relationship("ProposalChoices", back_populates="protocol_proposals")
    scores = relationship("ProposalScores", back_populates="protocol_proposals")

# ProtocolSpace class with a separate table for member elements
class ProtocolSpace(Base):
    __tablename__ = 'protocol_space'
    id = Column(String, primary_key=True)
    name = Column(String)
    avatar = Column(String)
    symbol = Column(String)
    verified = Column(Boolean)
    plugin = Column(JSON, nullable=True)
    space_id = Column(Integer)
    protocol_proposals = relationship("ProtocolProposals", back_populates="protocol_space")
    members = relationship("SpaceMember", back_populates="protocol_space")

# Create an SQLite database and tables
engine = create_engine('sqlite:///lido_snapshot.db')
Base.metadata.create_all(engine)

# [Continue with your script to fetch data and create DataFrames]

# Function to convert and format DataFrame for SQLite
def prepare_df_for_sqlite(df):
    for column in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[column]):
            df[column] = df[column].dt.strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(df[column].iloc[0], dict):
            df[column] = df[column].apply(json.dumps)
        else:
            df[column] = df[column].apply(str)
    return df

# Prepare proposal_votes_df for insertion
proposal_votes_df = prepare_df_for_sqlite(proposal_votes_df)

# Insert proposal_votes_df into SQLite table
try:
    proposal_votes_df.to_sql('lido_votes', engine, if_exists='replace', index=False)
    print("proposal_votes_df successfully inserted into 'lido_votes' table")
except Exception as e:
    print(f"An error occurred when inserting proposal_votes_df: {e}")

# Optional: Commit the changes
session = sessionmaker(bind=engine)()
session.commit()
session.close()

proposal_votes_df successfully inserted into 'lido_votes' table


/var/folders/v0/71tqdwzx1_11vzpcg1y_4mj80000gn/T/ipykernel_7085/2544316948.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
